`Co może pójść nie tak?

### 1. Monitorowanie nagłówków

Strony na różne sposoby są zabezpieczane przed scrapingiem. Najprostszym sposobem zabezpieczenia przed scrapingiem jest sprawdzanie nagłówków zapytania. W nagłówkach zapytania jest pole `User-Agent`, które przeglądarki wypełniają podając w nim swoje podstawowe parametry. Strony często sprawdzają, czy nagłówek `User-Agent` jest sensowny i jeżeli nie jest to odrzucają zapytanie. Jako przykład weźmy stronę Narodowego Banku Polskiego [NBP](https://www.nbp.pl/).

In [4]:
import requests

url = "https://nbp.pl/"
response = requests.get(url)
response.text

'<html>\r\n<head>\r\n<META NAME="robots" CONTENT="noindex,nofollow">\r\n<script src="/_Incapsula_Resource?SWJIYLWA=5074a744e2e3d891814e9a2dace20bd4,719d34d31c8e3a6e6fffd425f7e032f3">\r\n</script>\r\n<body>\r\n</body></html>\r\n'

Jakie nagłówki wysłaliśmy w zapytaniu?

Żeby to sprawdzić nie możemy posługiwać się metodą `get`, która tworzy i natychmiast wysyła zapytanie. Stwórzmy identyczne zapytanie ręcznie, przy pomocy klas `Session` i `Request`. W ten sposób będziemy mogli zobaczyć domyślne nagłówki, które biblioteka `requests` umieszcza w zapytaniu.

In [7]:
session = requests.Session()
request = requests.Request('GET', url)

prepared_request = session.prepare_request(request)
prepared_request.headers

{'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

Popatrzmy w przeglądarce jakie nagłówki dodaje.

Ten, na którym nam obecnie zależy to:

User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0

Co on oznacza? \
Sprawdźmy https://useragentstring.com/

Ustawmy go w naszym zapytaniu.

In [12]:
url = "https://nbp.pl/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
    "sec-ch-ua": '"Microsoft Edge";v="131", "Chromium";v="131", "Not_A Brand";v="24"'
}

response = requests.get(url, headers=headers)
content = response.text

Tym razem dostalismy prawidłową odpowiedź. Wyciągnijmy z niej kursy walut.

In [30]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(content, 'html.parser')
currencies = soup.find_all(class_="exchange__table-col-1")
rates = soup.find_all(class_="exchange__table-col-2")

for currency, rate in zip(currencies, rates):
    currency = currency.text.strip()
    rate = rate.text.strip()
    print(f"{currency}: {rate}")

1 USD: 4,0571
1 EUR: 4,2622
1 CHF: 4,5563
1 GBP: 5,1339
100 JPY: 2,6379


Jak to wygląda od strony prawnej?

Jeżeli dane są publicznie dostępne i nie zawierają wrażliwych informacji, to ich scraping jest legalny. W przypadku informacji, które uzyskać możemy dopiero po zalogowaniu się do określonego portalu ich scraping jest nielegalny. W tym przypadku najlepiej poszukać szczegółowych informacji na stronie portalu, ale w przeważającej liczbie przypadków scraping danych niepublicznych (do których uzyskania trzeba być uwierzytelnionym) jest nielegalny.

### 2. Monitorowanie IP

Standardowo monitorowana jest też liczba zapytań przychodzących z jednego ip w jednoste czasu. Jeżeli zapytania przychodzą szybciej niż jakikolwiek człowiek byłby w stanie wykonać ip często jest blokowany na tej stronie. Istnieją dwa popoularne sposoby na obchodzenie tego mechanizmu:
- dorzucanie `time.sleep` w taki sposób, żeby zapytania nie wychodziły zbyt często
- używanie proxy (minus: adresy publicznych proxy są powszechnie znane i często pojawiają się na blacklistach portali).